In [1]:
import os
os.environ['autocnet_config'] = '/home/jlaura/autocnet_projects/demo.yml'
from autocnet_server.utils.add_image import ImageAdder

%pylab inline

Populating the interactive namespace from numpy and matplotlib


## Adding images to the database

The ImageAdder object is a job dispatcher that has two primary access methods: `add_image` and `add_images`.  The former adds a single image to the database and the latter accepts both a directory and a file extension.  All files with the extension in said directory will be added to the database.  The process of adding an image includes the extraction of interest points (features), serialization of the interest points to disk in the same location as the image (saved as a .h5 file), generation of a camera model, computation of the image footprint, and addition of this information to the database.

When the object is created a message will print indicating that a PYRO server is also running:

```
Object <autocnet_server.utils.add_image.ImageAdder object at 0x2ae8b549b9e8>:
    uri = PYRO:ia@0.0.0.0:8005
Pyro daemon running.
```

This is entirely normal.  NAC works asynchronously, in that jobs are spawned to the cluster without locking up the jupyter notebook that can be used to work with NAC objects. The NAC objects create a listening server on a different thread and register callback functions so that, once a cluster job has completed, the NAC objects can be alerted.  This allows the NAC objects to manage the database connection and not swamp the database with hundreds of concurrent connections.

For now, the port (8005) is hard coded.  If the next cell is run twice, without restarting the jupyter kernel, an error will be thrown that port (8005) is already in use.  In the menus at the top of this notebook, select 'Kernel->Restart' to restart the kernel and free the port.

In [2]:
ia = ImageAdder()

Object <autocnet_server.utils.add_image.ImageAdder object at 0x2b3da173f438>:
    uri = PYRO:ia@0.0.0.0:8010
Pyro daemon running.


## Add Images
Here a directory of images is added to the database. The `add_images` methods takes two arguments: (1) the directory NS (2) the file extension.  All files in the directory that have the file extension will be added to the database table (defined) above in the `d={}` cell.

In [3]:
res = ia.add_images('/scratch/jlaura/ctx_demo/cal/', 'cub')

## Add a single Image
and force the keypoints to be reextracted

In [5]:
ia.add_image('/scratch/jlaura/ctx_demo/cal/B12_014304_1893_XN_09N203W.cal.cub', force=True)

## What is the image adder doing?

![extractorflow](../docs/images/extractor_flow.png)

- The extractor accepts either a directory of images and a file extension or a full path to a given image.  In the former case *all* images that match the pattern `*.extension` are addedd to the database that backs AutoCNet server.
- When images are added to the `ImageAdder` object number of cluster jobs equal to the number of images are launched and feature extraction occurs.
- The `ImageAdder` attempts feature extraction up to `maxfailures` (defined in the configuration file) time before truly failing.  This makes the `ImageAdder` redundant to cluster failures that are beyond our control.
- The figure above enumerate 'what' occurs inside each extraction process.

## Job Status

Knowing that a job has been submitted to a cluster is not all that helpful.  What matters is that the job completed successfully.  The `ia` object has two attributes to help with knowing what is going on with the jobs.  The first is `ia.job_status` that returns a dictionary with the key being the image name and the value being the either `True` (everything is working great and the image has be processed successfully) or a number (1,2,3,4).  If the value is a number, it means that the job has failed $n$ times so far.  We let a job fail 3 times before giving up on that image.

In [8]:
ia.job_status

defaultdict(dict, {})

The above is along list with (hopefully) lots of True values.  What is really of interest are the issues where keypoints could not be extracted.  To see those, the image_adder has a failed jobs attribute.

The output below is a little bit lame - an empty list indicating that everything worked.  Let's pretend for a minute that the list contained `'/scratch/jlaura/ctx/cal/B01_009900_1837_XN_03N206W.cal.cub'`.  We might want to inspect this job to see what happened.